#### Import the results as a pickle file

In [3]:
import pandas as pd 


df = pd.read_pickle("data/T500BaselineDual.pkl")
df = df[~df['Prompts'].str.contains("Manual")]
failed_rows, correct  = [], []
for index, row in df.iterrows():
    sexpgen = str(row['sexpgen'])

    if "VALUE" in sexpgen:
        correct.append(row)
    else:
        failed_rows.append(row)
    
df_failed = pd.DataFrame(failed_rows)
df_correct = pd.DataFrame(correct)


#### Table Exact Match

In [5]:

table_ids = []
counter = 0
for index, row in df_correct.iterrows():
    predicted_Table = str(row['sexpgen']).split("(VALUE (")[1].split(" ")[0]
    golden_table = row['table_id']

    if predicted_Table == golden_table:
        counter += 1

print("The table exact match is")
counter / len(df_correct)

The table exact match is


0.5353185595567868

#### Measure Exact Match

In [6]:
##Chcek the measures
correct_measures = 0
for index, row in df_correct[:].iterrows():
    sexpgen = str(row['sexpgen'])

    golden_measure = row['Measure'][0]

    generated = sexpgen.split("MSR")[1].split(" ")[1]
    # print(generated)
    if golden_measure == generated:
        correct_measures += 1

print("The measure em is")
correct_measures / len(df_correct)

The measure em is


0.3656509695290859

##### Rouge and blue

In [7]:
import re
import evaluate



to_skip_geo = ['Regio', 'RegioS', 'WijkenEnBuurten', 'Regiokenmerken']
to_skip_time = ['Perioden', "GN"]
bleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")


golden_sexps, generated_sexps = [], []
for index, row in df_correct[:].iterrows():
    # print(row['sexpgen'])
    generated = str(row['sexpgen'])
    ## GOlden Sexp
    golden_Sexp = row['original_sexp']
    # print(f"\n{generated}\n")

    generated = generated.replace("RegioS", "Regio")
    for skip in to_skip_geo:
        if skip in generated:
            # print("Skipping")
            
            generated = generated.replace(skip, "GC")

    for skip in to_skip_time:
        if skip in generated:
            # print("Skipping")
            
            generated = generated.replace(skip, "TC")
    generated = re.sub(r'GC \w+\b', 'GC <GC>', generated)
    generated = re.sub(r'TC \w+\b', 'TC <TC>', generated)

    golden_Sexp = re.sub(r'GC \w+\b', 'GC <GC>', golden_Sexp)
    golden_Sexp = re.sub(r'TC \w+\b', 'TC <TC>', golden_Sexp)

    golden_sexps.append(golden_Sexp)
    generated_sexps.append(generated)

rouge_score = rouge.compute(predictions=generated_sexps, references=golden_sexps,
                            rouge_types=["rouge2"])["rouge2"]
bleu_score = bleu.compute(predictions=generated_sexps, references=golden_sexps)["score"]

print(f"ROUGE-2: {round(rouge_score, 3)};\n"
      f"BLEU: {round(bleu_score, 3)};\n")

ROUGE-2: 0.578;
BLEU: 70.802;



#### DIM F1

In [9]:
import re
import numpy as np
# Initialize an empty list to store the F1 scores for each comparison
dim_f1 = []

# Regex pattern to find dimension groups and their IDs
pattern = r"\(DIM\s+([^\s]+)\s+([A-Z0-9_]+)\)"

for generated, golden in zip(generated_sexps, golden_sexps):
    # Find all matches for generated and golden S-expressions
    matches_generated = re.findall(pattern, generated)
    matches_golden = re.findall(pattern, golden)

    # Convert matches to sets of tuples for easy comparison
    inf_dims = set(matches_generated)
    target_dims = set(matches_golden)

    # Calculate Precision, Recall, and F1 score
    p = 0 if len(inf_dims) == 0 else len(inf_dims & target_dims) / len(inf_dims)
    r = 1 if len(target_dims) == 0 else len(inf_dims & target_dims) / len(target_dims)
    f1 = 0 if p + r == 0 else 2 * ((p * r) / (p + r))

    # Append the calculated F1 score to the list
    dim_f1.append(f1)

# To see the F1 scores
round(np.average(dim_f1), 3)

0.353

#### Overall Exact Match

In [10]:
import re
table_ids = []
counter = 0

pattern = r"\(DIM\s+([^\s]+)\s+([A-Z0-9_]+)\)"



for index, row in df_correct.iterrows():
    sexpgen = str(row['sexpgen'])

    predicted_Table = str(row['sexpgen']).split("(VALUE (")[1].split(" ")[0]
    golden_table = row['table_id']
    golden_measure = row['Measure'][0]

    generated = str(row['sexpgen'])
    golden_Sexp = row['original_sexp']
        
    generated = re.sub(r'GC \w+\b', 'GC <GC>', generated)
    generated = re.sub(r'TC \w+\b', 'TC <TC>', generated)
    
    golden_Sexp = re.sub(r'GC \w+\b', 'GC <GC>', golden_Sexp)
    golden_Sexp = re.sub(r'TC \w+\b', 'TC <TC>', golden_Sexp)

    if predicted_Table == golden_table:
        generatedmeasure = sexpgen.split("MSR")[1].split(" ")[1]
            
        if golden_measure == generatedmeasure:
            # counter += 1
            matches_generated = re.findall(pattern, generated)
            matches_golden = re.findall(pattern, golden_Sexp)

            ## matches generated is a list of tuples just make into one list continaing the elements of the tuples
            matches_generated = sorted([item for sublist in matches_generated for item in sublist])
            matches_golden = sorted([item for sublist in matches_golden for item in sublist])

            if matches_generated == matches_golden:
                counter += 1
            else:
                continue
        else:
            continue
    else:
        continue





# df_correct['predicted_Tables'] = table_ids
print("The table score is")
counter / len(df_correct)

The table score is


0.12673130193905818

##### OpenAI Relevancy Score

In [11]:
from transformers import GPT2Tokenizer
import json
import openai
import warnings
warnings.filterwarnings("ignore")
import os
from openai import OpenAI
openai.api_key = "ADD YOUR API KEY"
os.environ["OPENAI_API_KEY"] = "ADD YOUR API KEY"
client = OpenAI()


def exact_openai_token_count(text):
	# Initialize the GPT-2 tokenizer
	tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

	# Tokenize the text and count the number of tokens
	tokens = tokenizer.encode(text)

	return len(tokens)



def prompt_chatgpt(doc, model="gpt-3.5-turbo-0125"):


	template = f"""
	In response to the query: '{doc}', 

	- "RelevancyScore": A float between 0.0, and 1.0 indicating how well the table, measures, and dimensions match the query. A score of 1.0 indicates a perfect match.E valuate how well the table description fits the query. give a grade in the range, 0.0 for very bad, 0.5 for okay, 1.0 for perfect. You can ignore geographical locations and dates 


	"""


	input_tokens = exact_openai_token_count(template)
	input_cost = input_tokens / 1000 * 0.0005   # Cost per 1K tokens for input

	
	response = client.chat.completions.create(
		model=model,    
		response_format={"type": "json_object"},
		messages=[
			{"role": "system", "content": "  A Table question and answering system retrieved a table cell with a certain description, you are tasked to evaluate how well the retrieved table cell fits the input query. You return a grade in a json format."},
			{"role": "user", "content": template}
		],
		temperature=0.0,

	)
	output = json.loads(response.choices[0].message.content)
	output_tokens = exact_openai_token_count(f"{output}")
	output_cost = output_tokens / 1000 * 0.0015  # Cost per 1K tokens for output

	# Calculate the total cost by adding input and output costs
	total_cost = input_cost + output_cost

	# # Print the costs
	# print(f"Input tokens: {input_tokens}, Cost: ${input_cost:.4f}")
	# print(f"Output tokens: {output_tokens}, Cost: ${output_cost:.4f}")
	# print(f"Total cost: ${total_cost:.4f}")
	return output, total_cost

In [13]:
from tqdm import tqdm
import utils.utils as util
costs, outputs = [], []
for index, row in tqdm(df_correct[:1].iterrows()):
    try:
        pattern = r"\(DIM\s+([^\s]+)\s+([A-Z0-9_]+)\)"

        prompts = []

        sexp = row['sexpgen']
        query = row['query']

        measure = sexp.split("MSR")[1].split(" ")[1]
            
        dimensions = re.findall(pattern, sexp)
            
        prompt = util.expressiontoprompt(sexp, measure, dimensions)
            
        string = f"This is the Query: {query} \n This is the prompt: {prompt}"
        # string += "Evaluate how well the prompt fits the query. give a grade, 0.0 for very bad, 0.5 for okay, 1.0 for perfect. You can ignore geographical locations and dates."

        output, cost = prompt_chatgpt(string)
        # print(output)
        # print(cost)

        costs.append(cost)
        outputs.append(output)
    except:
        costs.append(0)
        outputs.append("Error")

df_correct['costs'] = costs
df_correct['outputs'] = outputs